<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s overal message(disaster or not?):
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [31]:
import tensorflow as tf
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [32]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-30 05:24:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-30 05:24:56 (45.0 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [33]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys

In [34]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-30 05:24:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.214.128, 173.194.215.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-08-30 05:24:56 (101 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [35]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [36]:
train_data = pd.read_csv('train.csv')
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [37]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [38]:
test_data = pd.read_csv('test.csv')
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [39]:
random_data = train_data.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
Anyone need a P/U tonight? I play Hybrid Slayer ps4 EU. HMU @Cod8sandscrims @EmpirikGaming @CoDAWScrims @4TP_KOTC @4TPFA @afterShock_Org : This is NOT Disaster tweet
------------------------------------

tweet number 2 :
I remember when I worked at Mcdonalds I use to be hours late because we used military time and I use to mess up when I had to be there :/ : This is NOT Disaster tweet
------------------------------------

tweet number 3 :
@suelinflower there is no words to describe the physical painthey ripped you apart while you screamed for dear lifeits like been engulfed : This is NOT Disaster tweet
------------------------------------

tweet number 4 :
@kshllcenterpri1 @Progress4Ohio burning down buildings what you mean like when you burnt down those black churches? : This is a Disaster tweet
------------------------------------

tweet number 5 :
Hear @DrFriedenCDC talk on how to avoid thousands of resistant infections/deaths in next 5 yrs: http://t.co/niV8x5Tbe0 #

## Extract validation data from train data:

In [40]:
train_data['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [41]:
train_data['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [42]:
train_data, val_data, train_label, val_label = train_test_split(train_data['text'].to_numpy(), train_data['target'].to_numpy(), 
                                                                test_size=0.15, 
                                                                random_state=42)

In [43]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [44]:
sum([len(tweet.split()) for tweet in train_data])

96401

In [45]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [46]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [47]:
text_vectorizer.adapt(train_data)

In [48]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorizer(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  736,    9,    1,    8,  161,  681,    5, 1732,   12,    0,
          0,    0,    0,    0])>

In [49]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorizer(random_tweet))

tweet:  Calgary news weather and traffic for August 5 * ~ 90 http://t.co/qBdRYXSGlC http://t.co/VZOd7qFFlv 


vectorized tweet:  tf.Tensor(
[ 534   56  252    7  724   10  452  184 1830    1    1    0    0    0
    0], shape=(15,), dtype=int64)


In [50]:
#unique words
unique_words = text_vectorizer.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['nkulw', 'njenga', 'nj36', 'nittys', 'nitroglycerin', 'nitishkumar', 'nite', 'nissannews', 'nissan', 'niqqa']


## Embedding:

In [51]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim = 32, input_length=None)

In [52]:
sample_embed = embedding(text_vectorizer(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 32), dtype=float32, numpy=
array([[-3.05602551e-02, -1.77825093e-02, -5.67483902e-03,
        -1.35690570e-02, -1.13930590e-02, -1.03388801e-02,
         3.90558504e-02, -3.70899215e-02,  3.06962617e-02,
         4.38923575e-02,  3.01342644e-02,  3.25052626e-02,
        -4.68346849e-02,  3.40811051e-02,  3.97244729e-02,
        -2.94649601e-02,  4.34853099e-02,  3.08811665e-04,
         4.52690758e-02,  4.22392525e-02, -1.30891576e-02,
         1.45544894e-02, -4.14742157e-03, -4.85996492e-02,
        -1.63892023e-02,  6.85701519e-03, -3.17860618e-02,
         1.82966106e-02,  4.58780266e-02, -1.75339095e-02,
         1.84208639e-02,  1.79042108e-02],
       [ 1.37268640e-02, -4.11048532e-05, -4.05843183e-03,
        -2.43161079e-02,  1.09399073e-02,  3.37903760e-02,
         4.50798906e-02, -9.24657658e-03,  2.42280625e-02,
         2.90109180e-02,  1.30916759e-03,  2.24550255e-02,
         1.95846073e-02, -3.32020670e-02, -4.81324680e-02,
         3.30413505e-0

## Analyze with Naive Bayes using TF-IDF:

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [54]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

model_0.fit(train_data,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [55]:
val_pred = model_0.predict(val_data)
val_pred

array([0, 0, 0, ..., 1, 0, 0])

In [56]:
val_label

array([1, 0, 1, ..., 1, 0, 0])

In [57]:
accuracy_score(val_label, val_pred)

0.7863397548161121

In [58]:
print(classification_report(val_label, val_pred))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       651
           1       0.85      0.61      0.71       491

    accuracy                           0.79      1142
   macro avg       0.80      0.76      0.77      1142
weighted avg       0.80      0.79      0.78      1142



## Analyze with Vanilla Neural Network:

In [110]:
from helper_functions import create_tensorboard_callback

In [195]:
inputs = tf.keras.layers.Input(shape= (1,), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)

# x = tf.keras.layers.Dense(10, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs,outputs, name='ANN')

In [196]:
model_1.summary()

Model: "ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 global_average_pooling1d_15  (None, 32)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_40 (Dense)            (None, 1)                 33        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable p

In [197]:
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [198]:
train_label.shape

(6471,)

In [199]:
train_data.shape

(6471,)

In [200]:
# !rm -rf './tensorboard/ANN'

In [201]:
model_1.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220830-064637
Epoch 1/10
203/203 [==============================] - 2s 8ms/step - loss: 0.5098 - accuracy: 0.8295 - val_loss: 0.5179 - val_accuracy: 0.8012
Epoch 2/10
203/203 [==============================] - 1s 6ms/step - loss: 0.3371 - accuracy: 0.9207 - val_loss: 0.4718 - val_accuracy: 0.8012
Epoch 3/10
203/203 [==============================] - 1s 6ms/step - loss: 0.2564 - accuracy: 0.9390 - val_loss: 0.4614 - val_accuracy: 0.8012
Epoch 4/10
203/203 [==============================] - 1s 6ms/step - loss: 0.2107 - accuracy: 0.9485 - val_loss: 0.4641 - val_accuracy: 0.7986
Epoch 5/10
203/203 [==============================] - 1s 6ms/step - loss: 0.1803 - accuracy: 0.9541 - val_loss: 0.4742 - val_accuracy: 0.7942
Epoch 6/10
203/203 [==============================] - 1s 6ms/step - loss: 0.1580 - accuracy: 0.9620 - val_loss: 0.4863 - val_accuracy: 0.7942
Epoch 7/10
203/203 [==============================] - 1s 6ms/step - loss: 0.1409 - 

In [202]:
model_1.evaluate(val_data,val_label)

36/36 [==============================] - 0s 2ms/step - loss: 0.5512 - accuracy: 0.7925


[0.5511842966079712, 0.792469322681427]

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'downtown area of Montreal on fire now', 
        'I need a blue car']

In [ ]:
model_0.predict_proba(tweets)

array([[0.20651409, 0.79348591],
       [0.37466913, 0.62533087],
       [0.70686811, 0.29313189]])

In [ ]:
model_0.predict(tweets)

array([1, 1, 0])

In [ ]:
model_1.predict(tweets)

array([[0.9243444 ],
       [0.9237472 ],
       [0.16474679]], dtype=float32)

## Retrieve the trained word embeddings:

In [301]:
weights = model_1.get_layer('embedding_1').get_weights()[0]
weights

array([[ 0.10406056, -0.20603265,  0.24701604, ..., -0.31445011,
        -0.12074793,  0.33640656],
       [ 0.3946096 ,  0.07241339,  0.32380885, ..., -0.22122942,
        -0.01287187,  0.1191934 ],
       [ 0.156348  , -0.15056393,  0.2805518 , ..., -0.3293898 ,
        -0.09204781,  0.343719  ],
       ...,
       [ 0.9130227 ,  0.9676143 ,  0.94822484, ...,  0.9137153 ,
         0.78165966,  0.44458365],
       [-0.18459198, -0.6469038 , -0.20274636, ..., -0.64426637,
        -0.5937154 , -0.43215936],
       [-0.18186294, -0.08517163, -0.23458543, ..., -0.31640187,
        -0.40168843, -0.07477575]], dtype=float32)

In [300]:
model_1_weights.shape

(10000, 32)

In [303]:
vocab = text_vectorizer.get_vocabulary()
vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [305]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [306]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Visualize the embeddings with Embedding Projector: